In [1]:
import torch
import numpy as np

In [2]:
sentence = 'hi hello'
target_s = 'i hello'
char_set = sorted(list(set(sentence)), key=str.lower)
print(char_set)

[' ', 'e', 'h', 'i', 'l', 'o']


In [3]:
char_dic = {c:i for i, c in enumerate(char_set)}
print(char_dic)

{' ': 0, 'e': 1, 'h': 2, 'i': 3, 'l': 4, 'o': 5}


In [4]:
input_size = len(char_set)
hidden_size = 7

In [5]:
learning_rate = 0.1

In [6]:
x_data = [char_dic[c] for c in sentence[:-1]]
print(x_data)

[2, 3, 0, 2, 1, 4, 4]


In [7]:
x_one_hot = [[np.eye(input_size)[c] for c in x_data]]
print(x_one_hot)

[[array([0., 0., 1., 0., 0., 0.]), array([0., 0., 0., 1., 0., 0.]), array([1., 0., 0., 0., 0., 0.]), array([0., 0., 1., 0., 0., 0.]), array([0., 1., 0., 0., 0., 0.]), array([0., 0., 0., 0., 1., 0.]), array([0., 0., 0., 0., 1., 0.])]]


In [8]:
y_data = [[char_dic[c] for c in target_s[:]]]
print(y_data)

[[3, 0, 2, 1, 4, 4, 5]]


In [9]:
x = torch.FloatTensor(x_one_hot)
y = torch.LongTensor(y_data)

print(x.shape)
print(y.shape)

torch.Size([1, 7, 6])
torch.Size([1, 7])


In [10]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True)
        self.flatten = torch.nn.Flatten()
        self.linear = torch.nn.Linear(hidden_size * 7, input_size, bias=True)
        
    def forward(self, x):
        x, _status = self.rnn(x)
#         x = self.flatten(x)
        x = x.reshape(x.size(0), -1)
        x = self.linear(x)
        
        return y
    
net = Net().to('cpu')

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), learning_rate)

In [11]:
x_chk = torch.randn((1, 7, 6))

net(x_chk)

tensor([[3, 0, 2, 1, 4, 4, 5]])

In [13]:
from torchsummary import summary
import torchsummary

summary(net, (7, 6), batch_size = 512)

---------------------------------------------------------------------------------------------------------------------------------------
        Layer (type)    Kernel Shape          Input Shape                        Output Shape              Param #          Multi Ops #
               RNN-1               -          [512, 7, 6]          [[512, 7, 7], [512, 2, 7]]                  105                   91
            Linear-2         [49, 6]            [512, 49]                            [512, 6]                  300                  294
Total params: 405
Trainable params: 405
Non-trainable params: 0
Multi-Adds params: 385
---------------------------------------------------------------------------------------------------------------------------------------
Input size (MB): 0.08
Forward/backward pass size (MB): 1372.02
Params size (MB): 0.00
Estimated Total Size (MB): 1372.11
----------------------------------------------------------------------------------------------------------------

(405, 405)

In [13]:
from torchsummaryX import summary

summary(net, torch.zeros(1, 7, 6))

         Kernel Shape Output Shape  Params  Mult-Adds
Layer                                                
0_rnn               -    [1, 7, 7]     105         91
1_linear      [49, 6]       [1, 6]     300        294
-----------------------------------------------------
                      Totals
Total params             405
Trainable params         405
Non-trainable params       0
Mult-Adds                385


In [14]:
for i in range(100):
    optimizer.zero_grad()
    outputs, _status = net(x)
    loss = criterion(outputs.view(-1, input_size), y.view(-1))
    loss.backward()
    optimizer.step()
    
    result = outputs.data.numpy().argmax(axis=2)
    result_str = ''.join([char_set[c] for c in np.squeeze(result)])
    print(i, loss.item(), result, y_data, result_str)

ValueError: not enough values to unpack (expected 2, got 1)